In [59]:
import cv2
img = cv2.imread('default_frame.jpg')

drawing = False
ix, iy = -1, -1
ex, ey = -1, -1

def draw_rectangle(event, x, y, flags, params):
    global ix, iy, drawing, ex, ey

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            cv2.rectangle(img, (ix, iy), (x, y), (0, 0, 255), -1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (ix, iy), (x, y), (0, 0, 255), -1)
        ex, ey = x, y


cv2.namedWindow(winname='get_border')
cv2.setMouseCallback('get_border', draw_rectangle)


while True:
    cv2.imshow('get_border', img)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

455
390
892
663


In [60]:
import cv2
import time

# Create a VideoCapture object to read from the video file
cap = cv2.VideoCapture('video_cap.mp4')

# Define the coordinates of the border region (left, top, right, bottom)
border_coordinates = (ix, iy, ex, ey)  # Adjust these coordinates as needed

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Couldn't open the video file.")
else:
    # Read the first frame from the video
    ret, prev_frame = cap.read()
    frame_count = 0

    # Check if the frame was read successfully
    if not ret:
        print("Error: Couldn't read the first frame.")
    else:
        while True:
            # Read the next frame from the video
            ret, current_frame = cap.read()
            real_current_frame = current_frame

            # If the frame was not read successfully, break the loop
            if not ret:
                break

            # Crop the frames to the specified border region
            prev_frame_cropped = prev_frame[border_coordinates[1]:border_coordinates[3], border_coordinates[0]:border_coordinates[2]]
            current_frame_cropped = current_frame[border_coordinates[1]:border_coordinates[3], border_coordinates[0]:border_coordinates[2]]

            # Compute the absolute difference between the cropped frames
            difference = cv2.absdiff(prev_frame_cropped, current_frame_cropped)
            threshold = 30

            # Convert the difference image to grayscale
            gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)

            print(cv2.countNonZero(gray_difference))
            
            # If a significant difference is detected, save the frame
            if cv2.countNonZero(gray_difference) >= 100000:

                # Apply a threshold to highlight the differences
                _, thresholded_diff = cv2.threshold(gray_difference, 30, 255, cv2.THRESH_BINARY)
                # Find contours in the thresholded difference image
                contours, _ = cv2.findContours(thresholded_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # Draw rectangles around the detected changes on the original image
                for contour in contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    # Adjust the coordinates to match the original frame
                    x += border_coordinates[0]
                    y += border_coordinates[1]
                    cv2.rectangle(prev_frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

                # Display the original image with changes highlighted
                imS = cv2.resize(prev_frame, (9602, 740))
                cv2.imshow('Difference Image', prev_frame_cropped)

                filename = f'diff_framenew_{frame_count}.jpg'
                cv2.imwrite(filename, prev_frame)

            frame_count += 1

            # Update the previous frame for the next iteration
            prev_frame = real_current_frame

            # If the user presses 'q', exit the loop
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        # Release the VideoCapture and close OpenCV windows
        cap.release()
        cv2.destroyAllWindows()


93829
90447
85806
81078
75398
65501
66205
62861
65119
64419
64935
65971
62539
64643
64361
65358
62223
64023
64133
63586
62602
62429
64115
63756
64401
63656
65127
67961
66594
103194
82113
75082
74868
74714
75610
65095
68227
68696
66944
62884
64966
61549
61667
61824
63489
64880
64009
64865
64278
65152
64076
65007
66242
63469
65712
62802
63159
64261
65046
102759
82627
72370
71034
75913
73328
66262
65428
65053
64109
63155
63229
63287
64297
63695
62719
64801
64914
64723
63974
63250
64224
64941
66556
64385
65215
64746
66065
63176
65362
102272
83210
71579
62741
63538
61492
64269
62534
61719
63451
63340
63323
64987
62429
65475
62235
66523
64887
63564
63456
64727
63700
62803
64634
65173
64979
66195
67941
68626
69022
105166
85145
77390
80048
77703
82025
82747
83924
85113
85640
84204
87523
82675
81920
83921
83228
79076
79694
81101
78756
78934
85906
89771
86011
85486
84375
85057
91552
92821
89970
108625
99270
100413
96078
95994
95251
96024
99980
101723
104270
103915
104954
102695
102095
103429
103

In [50]:
print(ix)
print(iy)
print(ex)
print(ey)

437
431
933
664
